In [2]:
# from two_TrAdaBoostR2 import TwoStageTrAdaBoostR2 ##STrAdaBoost.R2
# from TwoStageTrAdaBoostR2 import TwoStageTrAdaBoostR2 ##two-stage TrAdaBoost.R2

import pandas as pd
import sys
import numpy as np
from pandas import DataFrame
from sklearn.model_selection import train_test_split, KFold
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler 
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.cluster import KMeans

import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import seaborn as sns

from scipy.stats.stats import pearsonr
from math import sqrt
import statistics 
from scipy.stats import *
from scipy.spatial import distance

######### Instance Transfer repositories ####################
from adapt.instance_based import TrAdaBoost, TrAdaBoostR2, TwoStageTrAdaBoostR2
from sklearn.model_selection import GridSearchCV
from adapt.instance_based import KMM

print("Repositories uploaded!!")

Repositories uploaded!!


In [4]:
################################### Concrete ###########################################################################################################
ConcreteData_df = pd.read_excel('UCI_regression/Concrete/Concrete_Data.xls') ## 'Cement' found to be correlated at 0.4 :: 100
print("Concrete Data")
print("-------------------------------------------")
print(ConcreteData_df.shape)


##############################################################################################
# concrete_cols = ConcreteData_df.columns
# ss = StandardScaler()
# ConcreteData_df[concrete_cols] = ss.fit_transform(ConcreteData_df[concrete_cols])
# print(ConcreteData_df)


drop_col_concrete = ['Cement']

concrete_tgt_df = ConcreteData_df.loc[(ConcreteData_df['Cement'] <= 225)]
concrete_tgt_df = concrete_tgt_df.drop(drop_col_concrete, axis = 1)
concrete_tgt_df = concrete_tgt_df.reset_index(drop=True)
print("Target : ",concrete_tgt_df.shape)

concrete_source1_df = ConcreteData_df.loc[(ConcreteData_df['Cement'] > 225) & (ConcreteData_df['Cement'] <= 350)]
concrete_source1_df = concrete_source1_df.drop(drop_col_concrete, axis = 1)
concrete_source1_df = concrete_source1_df.reset_index(drop=True)
print("Source 1: ",concrete_source1_df.shape)

concrete_source2_df = ConcreteData_df.loc[(ConcreteData_df['Cement'] > 350)]
concrete_source2_df = concrete_source2_df.drop(drop_col_concrete, axis = 1)
concrete_source2_df = concrete_source2_df.reset_index(drop=True)
print("Source 2: ",concrete_source2_df.shape)

################################# Standardization ############################
from sklearn.preprocessing import StandardScaler
concrete_cols = concrete_tgt_df.columns.difference(['ConcreteCompressiveStrength'])

ss = StandardScaler()
concrete_tgt_df[concrete_cols] = ss.fit_transform(concrete_tgt_df[concrete_cols])
concrete_source1_df[concrete_cols] = ss.fit_transform(concrete_source1_df[concrete_cols])
concrete_source2_df[concrete_cols] = ss.fit_transform(concrete_source2_df[concrete_cols])


############################ Concatenating the source datasets ############################
concrete_source_df = pd.concat([concrete_source1_df, concrete_source2_df], ignore_index = True)
concrete_source_df = concrete_source_df.reset_index(drop = True)


#################### Splitting into features and target ####################
target_concrete = ['ConcreteCompressiveStrength']

concrete_source_df_y = concrete_source_df[target_concrete]
concrete_source_df_X = concrete_source_df.drop(target_concrete, axis = 1)

features_concrete = concrete_source_df_X.columns
################################################################################################################################################################

Concrete Data
-------------------------------------------
(1030, 9)
Target :  (368, 8)
Source 1:  (406, 8)
Source 2:  (256, 8)


In [5]:
########################### TwoStagetrAdaBoostR2 Concrete #######################################

from adapt.instance_based import TrAdaBoost, TrAdaBoostR2, TwoStageTrAdaBoostR2

def get_estimator(**kwargs):
    return DecisionTreeRegressor(max_depth = 6)

kwargs_TwoTrAda = {'steps': 30,
                    'fold': 10,
                  'learning_rate': 0.1}

print("Specification for TrAdaboost.R2 requirement complete!")

##################################################################################################

print("Two-Stage TrAdaboost.R2")
print("-------------------------------------------")

r2scorelist_TwoTrAda_concrete = []
rmselist_TwoTrAda_concrete = []

kf = KFold(n_splits = 20)

for train_idx, test_idx in kf.split(concrete_tgt_df):
        
    concrete_train_df_X = concrete_tgt_df.iloc[train_idx].loc[:, features_concrete]
    concrete_test_df_X = concrete_tgt_df.iloc[test_idx][features_concrete]
    concrete_train_df_y = concrete_tgt_df.iloc[train_idx].loc[:,target_concrete]
    concrete_test_df_y = concrete_tgt_df.loc[test_idx][target_concrete]
        
    concrete_X_df = pd.concat([concrete_source_df_X, concrete_train_df_X], ignore_index=True)
    concrete_y_df = pd.concat([concrete_source_df_y, concrete_train_df_y], ignore_index=True)

    concrete_np_train_X = concrete_X_df.to_numpy()
    concrete_np_train_y = concrete_y_df.to_numpy()

    concrete_np_test_X = concrete_test_df_X.to_numpy()
    concrete_np_test_y = concrete_test_df_y.to_numpy()

    concrete_np_train_y_list = concrete_np_train_y.ravel()
    concrete_np_test_y_list = concrete_np_test_y.ravel()

    src_size_concrete = len(concrete_source_df_y)
    tgt_size_concrete = len(concrete_train_df_y)
    
    src_idx = np.arange(start = 0, stop = (src_size_concrete - 1), step=1)
    tgt_idx = np.arange(start = src_size_concrete, stop = ((src_size_concrete + tgt_size_concrete)-1), step=1)


    model_TwoTrAda_concrete = TwoStageTrAdaBoostR2(get_estimator = get_estimator, n_estimators = 100) #, cv = 10) 
    model_TwoTrAda_concrete.fit(concrete_np_train_X, concrete_np_train_y_list, src_idx, tgt_idx)

    y_pred_TwoTrAda_concrete = model_TwoTrAda_concrete.predict(concrete_np_test_X)
    
    mse_TwoTrAda_concrete = sqrt(mean_squared_error(concrete_np_test_y, y_pred_TwoTrAda_concrete))
    rmselist_TwoTrAda_concrete.append(mse_TwoTrAda_concrete)
        
    r2_score_TwoTrAda_concrete = pearsonr(concrete_np_test_y_list, y_pred_TwoTrAda_concrete)
    r2_score_TwoTrAda_concrete = (r2_score_TwoTrAda_concrete[0])**2
    r2scorelist_TwoTrAda_concrete.append(r2_score_TwoTrAda_concrete)


print("RMSE List of Two-Stage TrAdaboost.R2:", rmselist_TwoTrAda_concrete)
print("R^2 List of TrAdaboostR2:", r2scorelist_TwoTrAda_concrete)

print("\n")

print("Mean, STDev of RMSE:", statistics.mean(rmselist_TwoTrAda_concrete), statistics.stdev(rmselist_TwoTrAda_concrete))
print("Mean, STDev of R^2:", statistics.mean(r2scorelist_TwoTrAda_concrete), statistics.stdev(r2scorelist_TwoTrAda_concrete))


print("-------------------------------------------")

Specification for TrAdaboost.R2 requirement complete!
Two-Stage TrAdaboost.R2
-------------------------------------------
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
RMSE List of Two-Stage TrAdaboost.R2: [6.541020044640769, 5.02012706206751, 9.169954145177607, 6.435689169158459, 8.88829865240887, 9.281962017215418, 8.875603050920281, 7.025704434153655, 5.13915260104503, 5.72072638391111, 7.369116818231316, 5.675806869960485, 5.75968959381744, 5.928300446961894, 6.428338521308748, 4.758416645321606, 4.5971373434502025, 6.883775231314941, 3.4293437208180224, 5.560616012498886]
R^2 List of TrAdaboostR2: [0.9026709048018587, 0.788561771700146, 0.4797770327

In [7]:
#################################### STrAdaBoost.R2 Active Sampling Concrete ################################################################
from two_TrAdaBoostR2 import TwoStageTrAdaBoostR2

n_estimators = 100
steps = 30
fold = 10

r2scorelist_stradaboost_concrete = []
rmselist_stradaboost_concrete = []

print("STrAdaboost.R2")
print("-------------------------------------------")


kf = KFold(n_splits = 20)

for train_idx, test_idx in kf.split(concrete_tgt_df):
        
    concrete_train_df_X = concrete_tgt_df.iloc[train_idx].loc[:, features_concrete]
    concrete_test_df_X = concrete_tgt_df.iloc[test_idx][features_concrete]
    concrete_train_df_y = concrete_tgt_df.iloc[train_idx].loc[:,target_concrete]
    concrete_test_df_y = concrete_tgt_df.loc[test_idx][target_concrete]
        
    concrete_X_df = pd.concat([concrete_source_df_X, concrete_train_df_X], ignore_index=True)
    concrete_y_df = pd.concat([concrete_source_df_y, concrete_train_df_y], ignore_index=True)

    concrete_np_train_X = concrete_X_df.to_numpy()
    concrete_np_train_y = concrete_y_df.to_numpy()

    concrete_np_test_X = concrete_test_df_X.to_numpy()
    concrete_np_test_y = concrete_test_df_y.to_numpy()

    concrete_np_train_y_list = concrete_np_train_y.ravel()
    concrete_np_test_y_list = concrete_np_test_y.ravel()
    
    sample_size = [len(concrete_source_df_X), len(concrete_train_df_X)]


    model_stradaboost_concrete = TwoStageTrAdaBoostR2(DecisionTreeRegressor(max_depth = 6),
                        n_estimators = n_estimators, sample_size = sample_size,
                        steps = steps, fold = fold)


    model_stradaboost_concrete.fit(concrete_np_train_X, concrete_np_train_y_list)
    y_pred_stradaboost_concrete = model_stradaboost_concrete.predict(concrete_np_test_X)


    mse_stradaboost_concrete = sqrt(mean_squared_error(concrete_np_test_y, y_pred_stradaboost_concrete))
    rmselist_stradaboost_concrete.append(mse_stradaboost_concrete)
        
    r2_score_stradaboost_concrete = pearsonr(concrete_np_test_y_list, y_pred_stradaboost_concrete)
    r2_score_stradaboost_concrete = (r2_score_stradaboost_concrete[0])**2
    r2scorelist_stradaboost_concrete.append(r2_score_stradaboost_concrete)


print("RMSE List of STrAdaboost.R2:", rmselist_stradaboost_concrete)
print("R^2 List of STrAdaboost.R2:", r2scorelist_stradaboost_concrete)

print("\n")

print("Mean, STDev of RMSE:", statistics.mean(rmselist_stradaboost_concrete), statistics.stdev(rmselist_stradaboost_concrete))
print("Mean, STDev of R^2:", statistics.mean(r2scorelist_stradaboost_concrete), statistics.stdev(r2scorelist_stradaboost_concrete))


STrAdaboost.R2
-------------------------------------------
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
RMSE List of STrAdaboost.R2: [6.160569688046794, 5.385056168151057, 11.537280214502667, 6.211790489243431, 10.124198380409222, 8.128429542667254, 6.394846310657595, 5.54877123897739

In [8]:
######################### Gradient Boosting Regression Transfer Learning Concrete #######################################

from sklearn.ensemble import GradientBoostingRegressor


print("Gradient Boosting Regression Transfer Learning")
print("-------------------------------------------")

r2scorelist_GBRTL_concrete = []
rmselist_GBRTL_concrete = []

kf = KFold(n_splits = 20)

for train_idx, test_idx in kf.split(concrete_tgt_df):
        
    concrete_train_df_X = concrete_tgt_df.iloc[train_idx].loc[:, features_concrete]
    concrete_test_df_X = concrete_tgt_df.iloc[test_idx][features_concrete]
    concrete_train_df_y = concrete_tgt_df.iloc[train_idx].loc[:,target_concrete]
    concrete_test_df_y = concrete_tgt_df.loc[test_idx][target_concrete]
        
    concrete_X_df = pd.concat([concrete_source_df_X, concrete_train_df_X], ignore_index=True)
    concrete_y_df = pd.concat([concrete_source_df_y, concrete_train_df_y], ignore_index=True)

    concrete_np_train_X = concrete_X_df.to_numpy()
    concrete_np_train_y = concrete_y_df.to_numpy()

    concrete_np_test_X = concrete_test_df_X.to_numpy()
    concrete_np_test_y = concrete_test_df_y.to_numpy()

    concrete_np_train_y_list = concrete_np_train_y.ravel()
    concrete_np_test_y_list = concrete_np_test_y.ravel()


    model_GBRTL_concrete = GradientBoostingRegressor(learning_rate = 0.1, max_depth = 6, n_estimators = 100)
    model_GBRTL_concrete.fit(concrete_np_train_X, concrete_np_train_y_list)

    y_pred_GBRTL_concrete = model_GBRTL_concrete.predict(concrete_test_df_X) ##Using dataframe instead of the numpy matrix
    
    mse_GBRTL_concrete = sqrt(mean_squared_error(concrete_np_test_y, y_pred_GBRTL_concrete))
    rmselist_GBRTL_concrete.append(mse_GBRTL_concrete)
        
    r2_score_GBRTL_concrete = pearsonr(concrete_np_test_y_list, y_pred_GBRTL_concrete)
    r2_score_GBRTL_concrete = (r2_score_GBRTL_concrete[0])**2
    r2scorelist_GBRTL_concrete.append(r2_score_GBRTL_concrete)


print("RMSE List of GBRTL:", rmselist_GBRTL_concrete)
print("R^2 List of GBRTL:", r2scorelist_GBRTL_concrete)

print("\n")

print("Mean, STDev of RMSE:", statistics.mean(rmselist_GBRTL_concrete), statistics.stdev(rmselist_GBRTL_concrete))
print("Mean, STDev of R^2:", statistics.mean(r2scorelist_GBRTL_concrete), statistics.stdev(r2scorelist_GBRTL_concrete))

print("-------------------------------------------")

Gradient Boosting Regression Transfer Learning
-------------------------------------------
RMSE List of GBRTL: [4.663742678225011, 6.126862455200775, 11.466485651569924, 5.534164994729643, 8.681619917565632, 6.642596890925768, 7.683746579901341, 5.739380797302863, 5.68449895901686, 4.182281620585482, 6.701558748703886, 2.215107778320667, 3.41335562523408, 2.7203268052979452, 3.7267350906136354, 2.616295028816559, 3.0657113984236566, 5.6021452410898105, 2.5329461221912757, 3.3591188620349595]
R^2 List of GBRTL: [0.9525246051803393, 0.6605331005165853, 0.38812568859308644, 0.8119852743703981, 0.7248962893294493, 0.8914405993092654, 0.6396109592952628, 0.39544149344564117, 0.7494443514243756, 0.9372748831817371, 0.74048501868772, 0.962487245469633, 0.9502436480826676, 0.9639690203712089, 0.893617787491994, 0.9216769812001803, 0.9012864535226115, 0.6609156893604556, 0.9439253841321719, 0.8892441889137216]


Mean, STDev of RMSE: 5.117934062287489 2.371938433398882
Mean, STDev of R^2: 0.7989

In [9]:
########################### AdaBoostR2 Transfer Learning Concrete #####################################################

from sklearn.ensemble import AdaBoostRegressor


print("Adaboost.R2 Transfer Learning")
print("-------------------------------------------")

r2scorelist_AdaTL_concrete = []
rmselist_AdaTL_concrete = []

kf = KFold(n_splits = 20)

for train_idx, test_idx in kf.split(concrete_tgt_df):
        
    concrete_train_df_X = concrete_tgt_df.iloc[train_idx].loc[:, features_concrete]
    concrete_test_df_X = concrete_tgt_df.iloc[test_idx][features_concrete]
    concrete_train_df_y = concrete_tgt_df.iloc[train_idx].loc[:,target_concrete]
    concrete_test_df_y = concrete_tgt_df.loc[test_idx][target_concrete]
        
    concrete_X_df = pd.concat([concrete_source_df_X, concrete_train_df_X], ignore_index=True)
    concrete_y_df = pd.concat([concrete_source_df_y, concrete_train_df_y], ignore_index=True)

    concrete_np_train_X = concrete_X_df.to_numpy()
    concrete_np_train_y = concrete_y_df.to_numpy()

    concrete_np_test_X = concrete_test_df_X.to_numpy()
    concrete_np_test_y = concrete_test_df_y.to_numpy()

    concrete_np_train_y_list = concrete_np_train_y.ravel()
    concrete_np_test_y_list = concrete_np_test_y.ravel()


    model_AdaTL_concrete = AdaBoostRegressor(DecisionTreeRegressor(max_depth = 6), learning_rate = 0.1, n_estimators = 100) 
    model_AdaTL_concrete.fit(concrete_np_train_X, concrete_np_train_y_list)
    
    y_pred_AdaTL_concrete = model_AdaTL_concrete.predict(concrete_np_test_X)
    
    mse_AdaTL_concrete = sqrt(mean_squared_error(concrete_np_test_y, y_pred_AdaTL_concrete))
    rmselist_AdaTL_concrete.append(mse_AdaTL_concrete)
        
    r2_score_AdaTL_concrete = pearsonr(concrete_np_test_y_list, y_pred_AdaTL_concrete)
    r2_score_AdaTL_concrete = (r2_score_AdaTL_concrete[0])**2
    r2scorelist_AdaTL_concrete.append(r2_score_AdaTL_concrete)
    

print("RMSE List of Adaboost.R2:", rmselist_AdaTL_concrete)
print("R^2 List of TrAdaboostR2:", r2scorelist_AdaTL_concrete)

print("\n")

print("Mean, STDev of RMSE:", statistics.mean(rmselist_AdaTL_concrete), statistics.stdev(rmselist_AdaTL_concrete))
print("Mean, STDev of R^2:", statistics.mean(r2scorelist_AdaTL_concrete), statistics.stdev(r2scorelist_AdaTL_concrete))


print("-------------------------------------------")

Adaboost.R2 Transfer Learning
-------------------------------------------
RMSE List of Adaboost.R2: [5.1351274696479905, 5.182085988671826, 11.198769903223992, 5.756563485271186, 7.717921817605795, 7.765110343491222, 7.759482005581707, 5.809474709537053, 4.438215821358044, 5.961850658965469, 7.656161242557137, 3.6574599776933203, 5.256006928900269, 4.785089971213188, 6.310064419585929, 5.8161850163181965, 5.530510893962602, 7.948659175138, 5.136579912083562, 6.6607104518557385]
R^2 List of TrAdaboostR2: [0.9445096800132503, 0.7609674496546639, 0.3739708899092762, 0.8080515694942868, 0.7622406718132485, 0.9156405848063586, 0.6221864736090811, 0.38647808741793743, 0.7017977235044947, 0.8669275860673343, 0.7201127063332123, 0.913618155294497, 0.8707740397739525, 0.8913360354288946, 0.6845594225426752, 0.7804855324959725, 0.6939375953484651, 0.5375468303906005, 0.8477889622106151, 0.6327007190586754]


Mean, STDev of RMSE: 6.274101509633112 1.6894400465595125
Mean, STDev of R^2: 0.73578153

In [70]:
from scipy import stats

stats_StAd_rmse, pval_StAd_rmse = stats.ttest_ind(rmselist_stradaboost_concrete, rmselist_AdaTL_concrete)
print(stats_StAd_rmse, pval_StAd_rmse)

stats_StGb_rmse, pval_StGb_rmse = stats.ttest_ind(rmselist_stradaboost_concrete, rmselist_GBRTL_concrete)
print(stats_StGb_rmse, pval_StGb_rmse)

stats_StTr_rmse, pval_StTr_rmse = stats.ttest_ind(rmselist_stradaboost_concrete, rmselist_TwoTrAda_concrete)
print(stats_StTr_rmse, pval_StTr_rmse)

print("\n")

stats_StAd_r2, pval_StAd_r2 = stats.ttest_ind(r2scorelist_stradaboost_concrete, r2scorelist_AdaTL_concrete)
print(stats_StAd_r2, pval_StAd_r2)

stats_StGb_r2, pval_StGb_r2 = stats.ttest_ind(r2scorelist_stradaboost_concrete, r2scorelist_GBRTL_concrete)
print(stats_StGb_r2, pval_StGb_r2)

stats_StTr_r2, pval_StTr_r2 = stats.ttest_ind(r2scorelist_stradaboost_concrete, r2scorelist_TwoTrAda_concrete)
print(stats_StTr_r2, pval_StTr_r2)

0.06642893536867998 0.9477686137441723
1.803370278742918 0.08809539363436718
-0.6402691951701012 0.5300643497257027


0.543419163483719 0.5935081302926962
-1.064876041637561 0.30100883616380025
1.3293754446871224 0.20032802775973285


In [75]:
#### Dependent(paired) t-test
from scipy.stats import ttest_rel


stats_StAd_rmse, pval_StAd_rmse = stats.ttest_rel(rmselist_stradaboost_concrete, rmselist_AdaTL_concrete)
print(stats_StAd_rmse, pval_StAd_rmse)

stats_StGb_rmse, pval_StGb_rmse = stats.ttest_rel(rmselist_stradaboost_concrete, rmselist_GBRTL_concrete)
print(stats_StGb_rmse, pval_StGb_rmse)

stats_StTr_rmse, pval_StTr_rmse = stats.ttest_rel(rmselist_stradaboost_concrete, rmselist_TwoTrAda_concrete)
print(stats_StTr_rmse, pval_StTr_rmse)

print("\n")

stats_StAd_r2, pval_StAd_r2 = stats.ttest_rel(r2scorelist_stradaboost_concrete, r2scorelist_AdaTL_concrete)
print(stats_StAd_r2, pval_StAd_r2)

stats_StGb_r2, pval_StGb_r2 = stats.ttest_rel(r2scorelist_stradaboost_concrete, r2scorelist_GBRTL_concrete)
print(stats_StGb_r2, pval_StGb_r2)

stats_StTr_r2, pval_StTr_r2 = stats.ttest_rel(r2scorelist_stradaboost_concrete, r2scorelist_TwoTrAda_concrete)
print(stats_StTr_r2, pval_StTr_r2)

0.2664013235666798 0.7959330728063094
2.3414918477916933 0.04391284579630366
-0.745171561376906 0.47517702097403447


2.137024794237644 0.06131719383991011
-2.2358256739080944 0.052197892656036644
2.702336607860335 0.024300531692972208


In [10]:
################################### Housing ################################################################
## 'nox' found to be correlated at 0.4 :: [0.385 - 0.871] :: 50
#################################################################################################################################
HousingData_df = pd.read_csv('UCI_regression/BostonHousing/BostonHousing.csv') 
print("Housing Data")
print(HousingData_df.shape)

drop_col_housing = ['nox']
housing_tgt_df = HousingData_df.loc[(HousingData_df['nox'] <= 0.475)]
housing_tgt_df = housing_tgt_df.drop(drop_col_housing, axis = 1)
housing_tgt_df = housing_tgt_df.reset_index(drop=True)
print("Target Set: ",housing_tgt_df.shape)


housing_source1_df = HousingData_df.loc[(HousingData_df['nox'] > 0.475) & (HousingData_df['nox'] <= 0.600)]
housing_source1_df = housing_source1_df.drop(drop_col_housing, axis = 1)
housing_source1_df = housing_source1_df.reset_index(drop=True)
print("Source Set 1: ",housing_source1_df.shape)


housing_source2_df = HousingData_df.loc[(HousingData_df['nox'] > 0.600)]
housing_source2_df = housing_source2_df.drop(drop_col_housing, axis = 1)
housing_source2_df = housing_source2_df.reset_index(drop=True)
print("Source Set 2: ",housing_source2_df.shape)

################################# Standardization ############################
from sklearn.preprocessing import StandardScaler
housing_cols = housing_tgt_df.columns.difference(['medv'])

ss = StandardScaler()
housing_tgt_df[housing_cols] = ss.fit_transform(housing_tgt_df[housing_cols])
housing_source1_df[housing_cols] = ss.fit_transform(housing_source1_df[housing_cols])
housing_source2_df[housing_cols] = ss.fit_transform(housing_source2_df[housing_cols])


############################ Concatenating the source datasets ############################
housing_source_df = pd.concat([housing_source1_df, housing_source2_df], ignore_index = True)
housing_source_df = housing_source_df.reset_index(drop = True)

#################### Splitting into features and target ####################
target_housing = ['medv']

housing_source_df_y = housing_source_df[target_housing]
housing_source_df_X = housing_source_df.drop(target_housing, axis = 1)

features_housing = housing_source_df_X.columns

Housing Data
(506, 14)
Target Set:  (155, 13)
Source Set 1:  (186, 13)
Source Set 2:  (165, 13)


In [11]:
########################### TwoStagetrAdaBoostR2 Housing #######################################

from adapt.instance_based import TrAdaBoost, TrAdaBoostR2, TwoStageTrAdaBoostR2

def get_estimator(**kwargs):
    return DecisionTreeRegressor(max_depth = 6)

kwargs_TwoTrAda = {'steps': 30,
                    'fold': 10,
                  'learning_rate': 0.1}

print("Specification requirement complete!")

###########################################################################################################################

print("Two-Stage TrAdaboost.R2")
print("-------------------------------------------")

r2scorelist_TwoTrAda_housing = []
rmselist_TwoTrAda_housing = []

kf = KFold(n_splits = 20)

for train_idx, test_idx in kf.split(housing_tgt_df):
        
    housing_train_df_X = housing_tgt_df.iloc[train_idx].loc[:, features_housing]
    housing_test_df_X = housing_tgt_df.iloc[test_idx][features_housing]
    housing_train_df_y = housing_tgt_df.iloc[train_idx].loc[:,target_housing]
    housing_test_df_y = housing_tgt_df.loc[test_idx][target_housing]
    
    housing_X_df = pd.concat([housing_source_df_X, housing_train_df_X], ignore_index = True)
    housing_y_df = pd.concat([housing_source_df_y, housing_train_df_y], ignore_index = True)

    housing_np_train_X = housing_X_df.to_numpy()
    housing_np_train_y = housing_y_df.to_numpy()

    housing_np_test_X = housing_test_df_X.to_numpy()
    housing_np_test_y = housing_test_df_y.to_numpy()

    housing_np_train_y_list = housing_np_train_y.ravel()
    housing_np_test_y_list = housing_np_test_y.ravel()

    src_size_housing = len(housing_source_df_y)
    tgt_size_housing = len(housing_train_df_y)
    
    src_idx_housing = np.arange(start = 0, stop = (src_size_housing - 1), step=1)
    tgt_idx_housing = np.arange(start = src_size_housing, stop = ((src_size_housing + tgt_size_housing)-1), step = 1)


    model_TwoTrAda_housing = TwoStageTrAdaBoostR2(get_estimator = get_estimator, n_estimators = 100) #, cv = 10) 
    model_TwoTrAda_housing.fit(housing_np_train_X, housing_np_train_y_list, src_idx_housing, tgt_idx_housing)

    y_pred_TwoTrAda_housing = model_TwoTrAda_housing.predict(housing_np_test_X)

    mse_TwoTrAda_housing = sqrt(mean_squared_error(housing_np_test_y, y_pred_TwoTrAda_housing))
    rmselist_TwoTrAda_housing.append(mse_TwoTrAda_housing)
        
    r2_score_TwoTrAda_housing = pearsonr(housing_np_test_y_list, y_pred_TwoTrAda_housing)
    r2_score_TwoTrAda_housing = (r2_score_TwoTrAda_housing[0])**2
    r2scorelist_TwoTrAda_housing.append(r2_score_TwoTrAda_housing)

print("RMSE List TrAdaboost.R2:", rmselist_TwoTrAda_housing)
print("R^2 List TrAdaboost.R2:", r2scorelist_TwoTrAda_housing)

print("\n")


print("Mean, STDev of RMSE:", statistics.mean(rmselist_TwoTrAda_housing), statistics.stdev(rmselist_TwoTrAda_housing))
print("Mean, STDev of R^2:", statistics.mean(r2scorelist_TwoTrAda_housing), statistics.stdev(r2scorelist_TwoTrAda_housing))


print("-------------------------------------------")

Specification requirement complete!
Two-Stage TrAdaboost.R2
-------------------------------------------
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.


In [12]:
#################################### STrAdaBoost.R2 Housing ################################################################
from two_TrAdaBoostR2 import TwoStageTrAdaBoostR2

n_estimators = 100
steps = 30
fold = 10
random_state = np.random.RandomState(1)

r2scorelist_stradaboost_housing = []
rmselist_stradaboost_housing = []

print("STrAdaboost.R2")
print("-------------------------------------------")

kf = KFold(n_splits = 20)

for train_idx, test_idx in kf.split(housing_tgt_df):
        
    housing_train_df_X = housing_tgt_df.iloc[train_idx].loc[:, features_housing]
    housing_test_df_X = housing_tgt_df.iloc[test_idx][features_housing]
    housing_train_df_y = housing_tgt_df.iloc[train_idx].loc[:,target_housing]
    housing_test_df_y = housing_tgt_df.loc[test_idx][target_housing]
    
    housing_X_df = pd.concat([housing_source_df_X, housing_train_df_X], ignore_index = True)
    housing_y_df = pd.concat([housing_source_df_y, housing_train_df_y], ignore_index = True)

    housing_np_train_X = housing_X_df.to_numpy()
    housing_np_train_y = housing_y_df.to_numpy()

    housing_np_test_X = housing_test_df_X.to_numpy()
    housing_np_test_y = housing_test_df_y.to_numpy()

    housing_np_train_y_list = housing_np_train_y.ravel()
    housing_np_test_y_list = housing_np_test_y.ravel()
    
    sample_size = [len(housing_source_df_X), len(housing_train_df_X)]


    model_stradaboost_housing = TwoStageTrAdaBoostR2(DecisionTreeRegressor(max_depth = 6),
                        n_estimators = n_estimators, sample_size = sample_size,
                        steps = steps, fold = fold, random_state = random_state)


    model_stradaboost_housing.fit(housing_np_train_X, housing_np_train_y_list)
    y_pred_stradaboost_housing = model_stradaboost_housing.predict(housing_np_test_X)


    mse_stradaboost_housing = sqrt(mean_squared_error(housing_np_test_y, y_pred_stradaboost_housing))
    rmselist_stradaboost_housing.append(mse_stradaboost_housing)
        
    r2_score_stradaboost_housing = pearsonr(housing_np_test_y_list, y_pred_stradaboost_housing)
    r2_score_stradaboost_housing = (r2_score_stradaboost_housing[0])**2
    r2scorelist_stradaboost_housing.append(r2_score_stradaboost_housing)

print("RMSE of STrAdaboost.R2:", rmselist_stradaboost_housing)
print("R^2 of STrAdaboost.R2:", r2scorelist_stradaboost_housing)

print("\n")


print("Mean, STDev of RMSE:", statistics.mean(rmselist_stradaboost_housing), statistics.stdev(rmselist_stradaboost_housing))
print("Mean, STDev of R^2:", statistics.mean(r2scorelist_stradaboost_housing), statistics.stdev(r2scorelist_stradaboost_housing))


STrAdaboost.R2
-------------------------------------------
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
RMSE of STrAdaboost.R2: [3.155749071986295, 2.4227148992447094, 1.512251171049698, 2.2651238680205252, 1.190147402947372, 1.849963794373305, 2.630304756303988, 2.950954565213517, 4.

In [13]:
########################### AdaBoostR2 Transfer Learning Housing #####################################################
from sklearn.ensemble import AdaBoostRegressor

print("Adaboost.R2 Transfer Learning")
print("-------------------------------------------")

r2scorelist_AdaTL_housing = []
rmselist_AdaTL_housing = []

kf = KFold(n_splits = 10)

for train_idx, test_idx in kf.split(housing_tgt_df):
        
    housing_train_df_X = housing_tgt_df.iloc[train_idx].loc[:, features_housing]
    housing_test_df_X = housing_tgt_df.iloc[test_idx][features_housing]
    housing_train_df_y = housing_tgt_df.iloc[train_idx].loc[:,target_housing]
    housing_test_df_y = housing_tgt_df.loc[test_idx][target_housing]
    
    housing_X_df = pd.concat([housing_source_df_X, housing_train_df_X], ignore_index = True)
    housing_y_df = pd.concat([housing_source_df_y, housing_train_df_y], ignore_index = True)

    housing_np_train_X = housing_X_df.to_numpy()
    housing_np_train_y = housing_y_df.to_numpy()

    housing_np_test_X = housing_test_df_X.to_numpy()
    housing_np_test_y = housing_test_df_y.to_numpy()

    housing_np_train_y_list = housing_np_train_y.ravel()
    housing_np_test_y_list = housing_np_test_y.ravel()


    model_AdaTL_housing = AdaBoostRegressor(DecisionTreeRegressor(max_depth = 6), learning_rate = 0.1, n_estimators = 100) 
    model_AdaTL_housing.fit(housing_np_train_X, housing_np_train_y_list)

    y_pred_AdaTL_housing = model_AdaTL_housing.predict(housing_np_test_X)

    mse_AdaTL_housing = sqrt(mean_squared_error(housing_np_test_y, y_pred_AdaTL_housing))
    rmselist_AdaTL_housing.append(mse_AdaTL_housing)
        
    r2_score_AdaTL_housing = pearsonr(housing_np_test_y_list, y_pred_AdaTL_housing)
    r2_score_AdaTL_housing = (r2_score_AdaTL_housing[0])**2
    r2scorelist_AdaTL_housing.append(r2_score_AdaTL_housing)

print("RMSE List of Adaboost.R2:", rmselist_AdaTL_housing)
print("R^2 List of AdaboostR2:", r2scorelist_AdaTL_housing)

print("\n")

print("Mean, Stdev of RMSE:", statistics.mean(rmselist_AdaTL_housing), statistics.stdev(rmselist_AdaTL_housing))
print("Mean, Stdev of R^2:", statistics.mean(r2scorelist_AdaTL_housing), statistics.stdev(r2scorelist_AdaTL_housing))


print("-------------------------------------------")

Adaboost.R2 Transfer Learning
-------------------------------------------
RMSE List of Adaboost.R2: [2.8789004616072678, 2.1031032183687057, 1.7656279207493897, 3.195696073337657, 6.7390127999478935, 1.9420006166548145, 7.491308432587034, 3.5319125801103444, 2.9373847752256412, 1.9416678345708085]
R^2 List of AdaboostR2: [0.8846718217146459, 0.9194323912559402, 0.6881629267471348, 0.9164251856498337, 0.7183945345658167, 0.8169316537644364, 0.5794145756627513, 0.9293708813008609, 0.7069307846681675, 0.9466234830058063]


Mean, Stdev of RMSE: 3.4526614713159556 2.027872057209755
Mean, Stdev of R^2: 0.8106358238335394 0.12861042495007238
-------------------------------------------


In [14]:
################################## Gradient Boosting Regression Transfer Learning Housing #######################################
from sklearn.ensemble import GradientBoostingRegressor

print("Gradient Boosting Regression Transfer Learning")
print("-------------------------------------------")

r2scorelist_GBRTL_housing = []
rmselist_GBRTL_housing = []

kf = KFold(n_splits = 10)

for train_idx, test_idx in kf.split(housing_tgt_df):
        
    housing_train_df_X = housing_tgt_df.iloc[train_idx].loc[:, features_housing]
    housing_test_df_X = housing_tgt_df.iloc[test_idx][features_housing]
    housing_train_df_y = housing_tgt_df.iloc[train_idx].loc[:,target_housing]
    housing_test_df_y = housing_tgt_df.loc[test_idx][target_housing]
    
    housing_X_df = pd.concat([housing_source_df_X, housing_train_df_X], ignore_index = True)
    housing_y_df = pd.concat([housing_source_df_y, housing_train_df_y], ignore_index = True)

    housing_np_train_X = housing_X_df.to_numpy()
    housing_np_train_y = housing_y_df.to_numpy()

    housing_np_test_X = housing_test_df_X.to_numpy()
    housing_np_test_y = housing_test_df_y.to_numpy()

    housing_np_train_y_list = housing_np_train_y.ravel()
    housing_np_test_y_list = housing_np_test_y.ravel()


    model_GBRTL_housing = GradientBoostingRegressor(learning_rate = 0.1, max_depth = 6, n_estimators = 100) #, subsample=0.5)
    model_GBRTL_housing.fit(housing_np_train_X, housing_np_train_y_list)

    y_pred_GBRTL_housing = model_GBRTL_housing.predict(housing_test_df_X) ##Using dataframe instead of the numpy matrix

    mse_GBRTL_housing = sqrt(mean_squared_error(housing_np_test_y, y_pred_GBRTL_housing))
    rmselist_GBRTL_housing.append(mse_GBRTL_housing)
        
    r2_score_GBRTL_housing = pearsonr(housing_np_test_y_list, y_pred_GBRTL_housing)
    r2_score_GBRTL_housing = (r2_score_GBRTL_housing[0])**2
    r2scorelist_GBRTL_housing.append(r2_score_GBRTL_housing)

print("RMSE List of GBRTL:", rmselist_GBRTL_housing)
print("R^2 List of GBRTL:", r2scorelist_GBRTL_housing)

print("\n")


print("Mean, Stdev of RMSE:", statistics.mean(rmselist_GBRTL_housing), statistics.stdev(rmselist_GBRTL_housing))
print("Mean, Stdev of R^2:", statistics.mean(r2scorelist_GBRTL_housing), statistics.stdev(r2scorelist_GBRTL_housing))


print("-------------------------------------------")

Gradient Boosting Regression Transfer Learning
-------------------------------------------
RMSE List of GBRTL: [3.2890385679986216, 2.653844979358322, 1.5127751441347033, 2.790489419460206, 6.599679964015708, 2.0928891043021487, 7.000055586361052, 2.6223303032868386, 2.521557519432606, 1.9752554798294435]
R^2 List of GBRTL: [0.8132234727415761, 0.7902579698445445, 0.7628401026163645, 0.9180290121890459, 0.7216492883019031, 0.8074669793700914, 0.46393064952423, 0.936183964535872, 0.7740900717562701, 0.9178866856684362]


Mean, Stdev of RMSE: 3.305791606817965 1.9069069671538574
Mean, Stdev of R^2: 0.7905558196548333 0.13619488203011404
-------------------------------------------


In [71]:
from scipy import stats

stats_StAd_rmse, pval_StAd_rmse = stats.ttest_ind(rmselist_stradaboost_housing, rmselist_AdaTL_housing)
print(stats_StAd_rmse, pval_StAd_rmse)

stats_StGb_rmse, pval_StGb_rmse = stats.ttest_ind(rmselist_stradaboost_housing, rmselist_GBRTL_housing)
print(stats_StGb_rmse, pval_StGb_rmse)

stats_StTr_rmse, pval_StTr_rmse = stats.ttest_ind(rmselist_stradaboost_housing, rmselist_TwoTrAda_housing)
print(stats_StTr_rmse, pval_StTr_rmse)

print("\n")

stats_StAd_r2, pval_StAd_r2 = stats.ttest_ind(r2scorelist_stradaboost_housing, r2scorelist_AdaTL_housing)
print(stats_StAd_r2, pval_StAd_r2)

stats_StGb_r2, pval_StGb_r2 = stats.ttest_ind(r2scorelist_stradaboost_housing, r2scorelist_GBRTL_housing)
print(stats_StGb_r2, pval_StGb_r2)

stats_StTr_r2, pval_StTr_r2 = stats.ttest_ind(r2scorelist_stradaboost_housing, r2scorelist_TwoTrAda_housing)
print(stats_StTr_r2, pval_StTr_r2)

-0.11320807483759392 0.9111186613495608
0.6142376111006888 0.5467442730953394
-0.7877168016462025 0.4411126634078205


-0.3577646135896259 0.7246802877655424
-0.6235793100410496 0.5407261834520463
1.9792126669704677 0.06329652761434805


In [74]:
#### Dependent(paired) t-test
from scipy.stats import ttest_rel


stats_StAd_rmse, pval_StAd_rmse = stats.ttest_rel(rmselist_stradaboost_housing, rmselist_AdaTL_housing)
print(stats_StAd_rmse, pval_StAd_rmse)

stats_StGb_rmse, pval_StGb_rmse = stats.ttest_rel(rmselist_stradaboost_housing, rmselist_GBRTL_housing)
print(stats_StGb_rmse, pval_StGb_rmse)

stats_StTr_rmse, pval_StTr_rmse = stats.ttest_rel(rmselist_stradaboost_housing, rmselist_TwoTrAda_housing)
print(stats_StTr_rmse, pval_StTr_rmse)

print("\n")

stats_StAd_r2, pval_StAd_r2 = stats.ttest_rel(r2scorelist_stradaboost_housing, r2scorelist_AdaTL_housing)
print(stats_StAd_r2, pval_StAd_r2)

stats_StGb_r2, pval_StGb_r2 = stats.ttest_rel(r2scorelist_stradaboost_housing, r2scorelist_GBRTL_housing)
print(stats_StGb_r2, pval_StGb_r2)

stats_StTr_r2, pval_StTr_r2 = stats.ttest_rel(r2scorelist_stradaboost_housing, r2scorelist_TwoTrAda_housing)
print(stats_StTr_r2, pval_StTr_r2)

-0.7996854083202125 0.44448619214735663
1.7669802852038805 0.11103400281068965
-1.7148762764672583 0.12050921388855486


-1.1582914763418435 0.2765565179145322
-0.9686871369054251 0.35801096964238777
2.7169621385318274 0.023725511885053154


In [15]:
######################################################## Automobile ################################################################
## horsepower column has correlation 0.4 :: [46 - 230] :: 30
#################################################################################################################################
dropcol_initial_auto = ['name']
AutoData_df = pd.read_csv('UCI_regression/MPG/Auto.csv') ## horsepower column has correlation 0.4 :: [46 - 230] :: 30
AutoData_df = AutoData_df.drop(dropcol_initial_auto, axis = 1)
print("The shape of the Input data is: ", AutoData_df.shape)

drop_col_auto = ['horsepower']

auto_tgt_df = AutoData_df.loc[(AutoData_df['horsepower'] <= 80)]
auto_tgt_df = auto_tgt_df.drop(drop_col_auto, axis = 1)
auto_tgt_df = auto_tgt_df.reset_index(drop=True)
print("Target Set: ",auto_tgt_df.shape)

auto_source1_df = AutoData_df.loc[(AutoData_df['horsepower'] > 80) & (AutoData_df['horsepower'] <= 110)]
auto_source1_df = auto_source1_df.drop(drop_col_auto, axis = 1)
auto_source1_df = auto_source1_df.reset_index(drop=True)
print("Source Set 1: ",auto_source1_df.shape)

auto_source2_df = AutoData_df.loc[(AutoData_df['horsepower'] > 110)]
auto_source2_df = auto_source2_df.drop(drop_col_auto, axis = 1)
auto_source2_df = auto_source2_df.reset_index(drop=True)
print("Source Set 2: ",auto_source2_df.shape)

################################# Standardization ############################
from sklearn.preprocessing import StandardScaler
auto_cols = auto_tgt_df.columns.difference(['medv'])

ss = StandardScaler()
auto_tgt_df[auto_cols] = ss.fit_transform(auto_tgt_df[auto_cols])
auto_source1_df[auto_cols] = ss.fit_transform(auto_source1_df[auto_cols])
auto_source2_df[auto_cols] = ss.fit_transform(auto_source2_df[auto_cols])

## Concatenating the source datasets
auto_source_df = pd.concat([auto_source1_df, auto_source2_df], ignore_index = True)
auto_source_df = auto_source_df.reset_index(drop = True)

#################### Splitting into features and target ####################
target_auto = ['mpg']

auto_source_df_y = auto_source_df[target_auto]
auto_source_df_X = auto_source_df.drop(target_auto, axis = 1)

features_auto = auto_source_df_X.columns

The shape of the Input data is:  (392, 8)
Target Set:  (119, 7)
Source Set 1:  (157, 7)
Source Set 2:  (116, 7)


In [16]:
########################### TwoStage-TrAdaBoostR2 Auto #######################################

from adapt.instance_based import TrAdaBoost, TrAdaBoostR2, TwoStageTrAdaBoostR2


def get_estimator(**kwargs):
    return DecisionTreeRegressor(max_depth = 6)

kwargs_TwoTrAda = {'steps': 30,
                    'fold': 10,
                  'learning_rate': 0.1}

###########################################################################################################################

print("Two-Stage TrAdaboost.R2")
print("-------------------------------------------")

r2scorelist_TwoTrAda_auto = []
rmselist_TwoTrAda_auto = []

kf = KFold(n_splits = 20)


for train_idx, test_idx in kf.split(auto_tgt_df):
        
    auto_train_df_X = auto_tgt_df.iloc[train_idx].loc[:, features_auto]
    auto_test_df_X = auto_tgt_df.iloc[test_idx][features_auto]
    auto_train_df_y = auto_tgt_df.iloc[train_idx].loc[:,target_auto]
    auto_test_df_y = auto_tgt_df.loc[test_idx][target_auto]
    
    auto_X_df = pd.concat([auto_source_df_X, auto_train_df_X], ignore_index=True)
    auto_y_df = pd.concat([auto_source_df_y, auto_train_df_y], ignore_index=True)

    auto_np_train_X = auto_X_df.to_numpy()
    auto_np_train_y = auto_y_df.to_numpy()

    auto_np_test_X = auto_test_df_X.to_numpy()
    auto_np_test_y = auto_test_df_y.to_numpy()

    auto_np_train_y_list = auto_np_train_y.ravel()
    auto_np_test_y_list = auto_np_test_y.ravel()

    src_size_auto = len(auto_source_df_y)
    tgt_size_auto = len(auto_train_df_y)
    
    src_idx_auto = np.arange(start=0, stop=(src_size_auto - 1), step=1)
    tgt_idx_auto = np.arange(start=src_size_auto, stop=((src_size_auto + tgt_size_auto)-1), step=1)


    model_TwoTrAda_auto = TwoStageTrAdaBoostR2(get_estimator = get_estimator, n_estimators = 100) #, cv = 10) 
    model_TwoTrAda_auto.fit(auto_np_train_X, auto_np_train_y_list, src_idx_auto, tgt_idx_auto)

    y_pred_TwoTrAda_auto = model_TwoTrAda_auto.predict(auto_np_test_X)

    mse_TwoTrAda_auto = sqrt(mean_squared_error(auto_np_test_y, y_pred_TwoTrAda_auto))
    rmselist_TwoTrAda_auto.append(mse_TwoTrAda_auto)
        
    r2_score_TwoTrAda_auto = pearsonr(auto_np_test_y_list, y_pred_TwoTrAda_auto)
    r2_score_TwoTrAda_auto = (r2_score_TwoTrAda_auto[0])**2
    r2scorelist_TwoTrAda_auto.append(r2_score_TwoTrAda_auto)


print("RMSE List of TrAdaboost.R2:", rmselist_TwoTrAda_auto)
print("R^2 List of TrAdaboost.R2:", r2scorelist_TwoTrAda_auto)

print("\n")

print("Mean, Stdev of RMSE:", statistics.mean(rmselist_TwoTrAda_auto), statistics.stdev(rmselist_TwoTrAda_auto))
print("Mean, Stdev of R^2:", statistics.mean(r2scorelist_TwoTrAda_auto), statistics.stdev(r2scorelist_TwoTrAda_auto))


print("-------------------------------------------")

Two-Stage TrAdaboost.R2
-------------------------------------------
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set

In [17]:
#################################### STrAdaBoost.R2 Active Sampling Auto ################################################################
from two_TrAdaBoostR2 import TwoStageTrAdaBoostR2

n_estimators = 100
steps = 30
fold = 10
random_state = np.random.RandomState(1)

r2scorelist_stradaboost_auto = []
rmselist_stradaboost_auto = []

print("Two-Stage STrAdaboost.R2")
print("-------------------------------------------")

kf = KFold(n_splits = 20)


for train_idx, test_idx in kf.split(auto_tgt_df):

    auto_train_df_X = auto_tgt_df.iloc[train_idx].loc[:, features_auto]
    auto_test_df_X = auto_tgt_df.iloc[test_idx][features_auto]
    auto_train_df_y = auto_tgt_df.iloc[train_idx].loc[:,target_auto]
    auto_test_df_y = auto_tgt_df.loc[test_idx][target_auto]

    auto_X_df = pd.concat([auto_source_df_X, auto_train_df_X], ignore_index=True)
    auto_y_df = pd.concat([auto_source_df_y, auto_train_df_y], ignore_index=True)

    auto_np_train_X = auto_X_df.to_numpy()
    auto_np_train_y = auto_y_df.to_numpy()

    auto_np_test_X = auto_test_df_X.to_numpy()
    auto_np_test_y = auto_test_df_y.to_numpy()

    auto_np_train_y_list = auto_np_train_y.ravel()
    auto_np_test_y_list = auto_np_test_y.ravel()

    sample_size = [len(auto_source_df_X), len(auto_train_df_X)]


    model_stradaboost_auto = TwoStageTrAdaBoostR2(DecisionTreeRegressor(max_depth = 6),
                        n_estimators = n_estimators, sample_size = sample_size,
                        steps = steps, fold = fold, random_state = random_state)


    model_stradaboost_auto.fit(auto_np_train_X, auto_np_train_y_list)
    y_pred_stradaboost_auto = model_stradaboost_auto.predict(auto_np_test_X)


    mse_stradaboost_auto = sqrt(mean_squared_error(auto_np_test_y, y_pred_stradaboost_auto))
    rmselist_stradaboost_auto.append(mse_stradaboost_auto)

    r2_score_stradaboost_auto = pearsonr(auto_np_test_y_list, y_pred_stradaboost_auto)
    r2_score_stradaboost_auto = (r2_score_stradaboost_auto[0])**2
    r2scorelist_stradaboost_auto.append(r2_score_stradaboost_auto)


print("RMSE List of STrAdaboostR2:", rmselist_stradaboost_auto)
print("R^2 List of STrAdaboostR2:", r2scorelist_stradaboost_auto)

print("\n")

print("Mean, Stdev of RMSE:", statistics.mean(rmselist_stradaboost_auto), statistics.stdev(rmselist_stradaboost_auto))
print("Mean, Stdev of R^2:", statistics.mean(r2scorelist_stradaboost_auto), statistics.stdev(r2scorelist_stradaboost_auto))


print("-------------------------------------------")

Two-Stage STrAdaboost.R2
-------------------------------------------
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
RMSE List of STrAdaboostR2: [0.20601948811461177, 0.5049190379216697, 0.4229154213029683, 0.38813387023892226, 0.44945889021333246, 0.3179378362878423, 0.3883707713056787,

In [18]:
########################### AdaBoostR2 Transfer Learning Auto #####################################################

from sklearn.ensemble import AdaBoostRegressor


print("Adaboost.R2 Transfer Learning")
print("-------------------------------------------")

r2scorelist_AdaTL_auto = []
rmselist_AdaTL_auto = []

kf = KFold(n_splits = 20)

for train_idx, test_idx in kf.split(auto_tgt_df):
        
    auto_train_df_X = auto_tgt_df.iloc[train_idx].loc[:, features_auto]
    auto_test_df_X = auto_tgt_df.iloc[test_idx][features_auto]
    auto_train_df_y = auto_tgt_df.iloc[train_idx].loc[:,target_auto]
    auto_test_df_y = auto_tgt_df.loc[test_idx][target_auto]
    
    auto_X_df = pd.concat([auto_source_df_X, auto_train_df_X], ignore_index = True)
    auto_y_df = pd.concat([auto_source_df_y, auto_train_df_y], ignore_index = True)

    auto_np_train_X = auto_X_df.to_numpy()
    auto_np_train_y = auto_y_df.to_numpy()

    auto_np_test_X = auto_test_df_X.to_numpy()
    auto_np_test_y = auto_test_df_y.to_numpy()

    auto_np_train_y_list = auto_np_train_y.ravel()
    auto_np_test_y_list = auto_np_test_y.ravel()


    model_AdaTL_auto = AdaBoostRegressor(DecisionTreeRegressor(max_depth = 6), learning_rate = 0.1, n_estimators = 100) 
    model_AdaTL_auto.fit(auto_np_train_X, auto_np_train_y_list)

    y_pred_AdaTL_auto = model_AdaTL_auto.predict(auto_np_test_X)

    mse_AdaTL_auto = sqrt(mean_squared_error(auto_np_test_y, y_pred_AdaTL_auto))
    rmselist_AdaTL_auto.append(mse_AdaTL_auto)
        
    r2_score_AdaTL_auto = pearsonr(auto_np_test_y_list, y_pred_AdaTL_auto)
    r2_score_AdaTL_auto = (r2_score_AdaTL_auto[0])**2
    r2scorelist_AdaTL_auto.append(r2_score_AdaTL_auto)

print("RMSE List of TL Adaboost.R2:", rmselist_AdaTL_auto)
print("R^2 List of TL Adaboost.R2:", r2scorelist_AdaTL_auto)

print("\n")


print("Mean, Stdev of RMSE:", statistics.mean(rmselist_AdaTL_auto), statistics.stdev(rmselist_AdaTL_auto))
print("Mean, Stdev of R^2:", statistics.mean(r2scorelist_AdaTL_auto), statistics.stdev(r2scorelist_AdaTL_auto))


print("-------------------------------------------")

Adaboost.R2 Transfer Learning
-------------------------------------------
RMSE List of TL Adaboost.R2: [0.22748107275225346, 0.5576866535916815, 0.4434246830711074, 0.44206911587955655, 0.44483637515108765, 0.3229293484696365, 0.4238731741077402, 0.1643797897655724, 0.3938253176679665, 1.187062847771504, 0.256953574454448, 0.4847109431499351, 0.8007023507442278, 1.0764806524633945, 1.2786277877685757, 0.7076184233964553, 0.2433111282945678, 0.2900277664715691, 0.46951150461866903, 0.5020503434296196]
R^2 List of TL Adaboost.R2: [0.9341449124972279, 0.8093828756318796, 0.7571990301901858, 0.5733886260615008, 0.8831872731460747, 0.9368324073747525, 0.9076622134657332, 0.8184045492403138, 0.8837529099206358, 0.00422453415910673, 0.4954471946404811, 0.8145473733206147, 0.004014109009267995, 0.8630139365768354, 0.006484423736816256, 0.27765905884818776, 0.6955908048274713, 0.622117222775029, 0.022235180961524333, 0.7850115308942692]


Mean, Stdev of RMSE: 0.5358781426509784 0.31928398542180

In [19]:
######################### Gradient Boosting Regression Transfer Learning Auto #######################################
from sklearn.ensemble import GradientBoostingRegressor

print("Gradient Boosting Regression Transfer Learning")
print("-------------------------------------------")

r2scorelist_GBRTL_auto = []
rmselist_GBRTL_auto = []

kf = KFold(n_splits = 20)


for train_idx, test_idx in kf.split(auto_tgt_df):
        
    auto_train_df_X = auto_tgt_df.iloc[train_idx].loc[:, features_auto]
    auto_test_df_X = auto_tgt_df.iloc[test_idx][features_auto]
    auto_train_df_y = auto_tgt_df.iloc[train_idx].loc[:,target_auto]
    auto_test_df_y = auto_tgt_df.loc[test_idx][target_auto]
    
    auto_X_df = pd.concat([auto_source_df_X, auto_train_df_X], ignore_index = True)
    auto_y_df = pd.concat([auto_source_df_y, auto_train_df_y], ignore_index = True)

    auto_np_train_X = auto_X_df.to_numpy()
    auto_np_train_y = auto_y_df.to_numpy()

    auto_np_test_X = auto_test_df_X.to_numpy()
    auto_np_test_y = auto_test_df_y.to_numpy()

    auto_np_train_y_list = auto_np_train_y.ravel()
    auto_np_test_y_list = auto_np_test_y.ravel()


    model_GBRTL_auto = GradientBoostingRegressor(learning_rate = 0.1, max_depth = 6, n_estimators = 100) #, subsample=0.5)
    model_GBRTL_auto.fit(auto_np_train_X, auto_np_train_y_list)

    y_pred_GBRTL_auto = model_GBRTL_auto.predict(auto_test_df_X) ##Using dataframe instead of the numpy matrix

    mse_GBRTL_auto = sqrt(mean_squared_error(auto_np_test_y, y_pred_GBRTL_auto))
    rmselist_GBRTL_auto.append(mse_GBRTL_auto)
        
    r2_score_GBRTL_auto = pearsonr(auto_np_test_y_list, y_pred_GBRTL_auto)
    r2_score_GBRTL_auto = (r2_score_GBRTL_auto[0])**2
    r2scorelist_GBRTL_auto.append(r2_score_GBRTL_auto)

print("RMSE List of GBRTL:", rmselist_GBRTL_auto)
print("R^2 List of GBRTL:", r2scorelist_GBRTL_auto)

print("\n")

    
print("Mean, RMSE of GBRTL:", statistics.mean(rmselist_GBRTL_auto), statistics.stdev(rmselist_GBRTL_auto))
print("R^2 of GBRTL:", statistics.mean(r2scorelist_GBRTL_auto), statistics.stdev(r2scorelist_GBRTL_auto))


print("-------------------------------------------")

Gradient Boosting Regression Transfer Learning
-------------------------------------------
RMSE List of GBRTL: [0.15859050655125667, 0.41139005516563704, 0.30534265540932753, 0.3447435015940725, 0.37593585445404726, 0.30058815512729337, 0.47956632866938126, 0.0881958643456524, 0.4508185245736514, 1.0827960115053572, 0.4106504102376164, 0.4562080938028887, 0.7968386059635709, 0.9670057997290188, 1.353459021087352, 0.5604557385836028, 0.2779736500449244, 0.3488467042083944, 0.4280191784036677, 0.5927629547706863]
R^2 List of GBRTL: [0.9504214685443013, 0.869304169200463, 0.8110095747388791, 0.6249385351833867, 0.8731085756977781, 0.9192737616740614, 0.931551153149705, 0.9583856058811583, 0.41528569121741915, 0.005750062660484827, 0.32106536766060956, 0.6802353754517383, 0.0008452550349354726, 0.9472766821416176, 0.005253722706203909, 0.43078523899528703, 0.6565508881948792, 0.37358119478626134, 0.02946711689219825, 0.6713787892447104]


Mean, RMSE of GBRTL: 0.50950938071137 0.31505320532

In [72]:
#### Independent t-test
from scipy import stats

stats_StAd_rmse, pval_StAd_rmse = stats.ttest_ind(rmselist_stradaboost_auto, rmselist_AdaTL_auto)
print(stats_StAd_rmse, pval_StAd_rmse)

stats_StGb_rmse, pval_StGb_rmse = stats.ttest_ind(rmselist_stradaboost_auto, rmselist_GBRTL_auto)
print(stats_StGb_rmse, pval_StGb_rmse)

stats_StTr_rmse, pval_StTr_rmse = stats.ttest_ind(rmselist_stradaboost_auto, rmselist_TwoTrAda_auto)
print(stats_StTr_rmse, pval_StTr_rmse)

print("\n")

stats_StAd_r2, pval_StAd_r2 = stats.ttest_ind(r2scorelist_stradaboost_auto, r2scorelist_AdaTL_auto)
print(stats_StAd_r2, pval_StAd_r2)

stats_StGb_r2, pval_StGb_r2 = stats.ttest_ind(r2scorelist_stradaboost_auto, r2scorelist_GBRTL_auto)
print(stats_StGb_r2, pval_StGb_r2)

stats_StTr_r2, pval_StTr_r2 = stats.ttest_ind(r2scorelist_stradaboost_auto, r2scorelist_TwoTrAda_auto)
print(stats_StTr_r2, pval_StTr_r2)

0.09387925740000946 0.9262421791063029
0.468273988433133 0.6452092125474611
-0.7005342904293791 0.4925503891649965


-0.21561187287426226 0.8317146184323403
-0.08862966216384667 0.9303548911060949
0.7542751148496798 0.46043871525449653


In [73]:
#### Dependent(paired) t-test
from scipy.stats import ttest_rel

stats_StAd_rmse, pval_StAd_rmse = stats.ttest_rel(rmselist_stradaboost_auto, rmselist_AdaTL_auto)
print(stats_StAd_rmse, pval_StAd_rmse)

stats_StGb_rmse, pval_StGb_rmse = stats.ttest_rel(rmselist_stradaboost_auto, rmselist_GBRTL_auto)
print(stats_StGb_rmse, pval_StGb_rmse)

stats_StTr_rmse, pval_StTr_rmse = stats.ttest_rel(rmselist_stradaboost_auto, rmselist_TwoTrAda_auto)
print(stats_StTr_rmse, pval_StTr_rmse)

print("\n")

stats_StAd_r2, pval_StAd_r2 = stats.ttest_rel(r2scorelist_stradaboost_auto, r2scorelist_AdaTL_auto)
print(stats_StAd_r2, pval_StAd_r2)

stats_StGb_r2, pval_StGb_r2 = stats.ttest_rel(r2scorelist_stradaboost_auto, r2scorelist_GBRTL_auto)
print(stats_StGb_r2, pval_StGb_r2)

stats_StTr_r2, pval_StTr_r2 = stats.ttest_rel(r2scorelist_stradaboost_auto, r2scorelist_TwoTrAda_auto)
print(stats_StTr_r2, pval_StTr_r2)

1.027595320206241 0.33096466564742744
2.3507126938039717 0.04325455224503656
-1.4188348219534268 0.1896360652668623


-1.5799623613158462 0.14857296728108557
-0.55014538292201 0.5956072149748616
1.6768498418527644 0.12788653793474053


In [ ]:
import matplotlib.pyplot as plt


labels = ['Concrete', 'Housing', 'Auto']

TR_means = [20, 35, 30]
STR_means = [25, 32, 34]
ADA_means = [25, 32, 34]
GBR_means = [25, 32, 34]

TR_std = [20, 35, 30]
STR_std = [25, 32, 34]
ADA_std = [25, 32, 34]
GBR_std = [25, 32, 34]

width = 0.35       

fig, ax = plt.subplots()

ax.bar(labels, men_means, width, yerr=men_std, label='Men')
ax.bar(labels, women_means, width, yerr=women_std, bottom=men_means,
       label='Women')

ax.set_ylabel('Scores')
ax.set_title('Scores by group and gender')
ax.legend()

plt.show()